In [1]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
import time
import itertools
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
import gym_Aircraft

In [2]:
env = gym.make("acav-v0")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/jupyter-kunwoopark/.local/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [3]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """transition 저장"""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:
class DQN(nn.Module):
    def __init__(self, num_inputs, num_actions):
        super(DQN, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(env.observation_space.shape[0], 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, env.action_space.n)
        )
        
    def forward(self, x):
        return self.layers(x)

In [5]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

In [6]:
policy_net = nn.DataParallel(DQN(env.observation_space.shape[0], env.action_space.n).to(device))
target_net = nn.DataParallel(DQN(env.observation_space.shape[0], env.action_space.n).to(device))
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

DataParallel(
  (module): DQN(
    (layers): Sequential(
      (0): Linear(in_features=5, out_features=40, bias=True)
      (1): ReLU()
      (2): Linear(in_features=40, out_features=40, bias=True)
      (3): ReLU()
      (4): Linear(in_features=40, out_features=40, bias=True)
      (5): ReLU()
      (6): Linear(in_features=40, out_features=40, bias=True)
      (7): ReLU()
      (8): Linear(in_features=40, out_features=40, bias=True)
      (9): ReLU()
      (10): Linear(in_features=40, out_features=40, bias=True)
      (11): ReLU()
      (12): Linear(in_features=40, out_features=40, bias=True)
      (13): ReLU()
      (14): Linear(in_features=40, out_features=3, bias=True)
    )
  )
)

In [7]:
n_actions = env.action_space.n

In [8]:
optimizer = optim.Adam(policy_net.parameters())
memory = ReplayMemory(10000)

In [9]:
steps_done = 0

In [10]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max (1)은 각 행의 가장 큰 열 값을 반환합니다.
            # 최대 결과의 두번째 열은 최대 요소의 주소값이므로,
            # 기대 보상이 더 큰 행동을 선택할 수 있습니다.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

In [11]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)

    batch = Transition(*zip(*transitions))

    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = policy_net(state_batch).gather(1, action_batch)

    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # 기대 Q 값 계산
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Huber 손실 계산
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # 모델 최적화
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [ ]:
num_episodes = 5000
total_res=[]
reward_list=[]
for i_episode in range(num_episodes):
    total_reward=0
    
    # 환경과 상태 초기화
    res_list=np.zeros(11)
    state = env.reset()
    state=torch.from_numpy(state.astype(np.float32)).unsqueeze(0).to(device)
    for t in count():
        # 행동 선택과 수행
        
        action = select_action(state)
        next_state, reward, done, res = env.step(action.item())
        reward = torch.tensor([reward], dtype=torch.float32).to(device)
        
        next_state=torch.from_numpy(next_state.astype(np.float32)).unsqueeze(0).to(device)

        # 새로운 상태 관찰
        if not done:
            next_state = next_state
        else:
            next_state = None

        # 메모리에 변이 저장
        memory.push(state, action, next_state, reward)

        # 다음 상태로 이동
        state = next_state

        # 최적화 한단계 수행(목표 네트워크에서)
        optimize_model()
        
        # Data save
        
        cmd_list,r_list,elev_list,azim_list,Pm_list,Pt_list,h_list=res
        Pm_list=Pm_list.tolist()
        Pt_list=Pt_list.tolist()
        merged_data=itertools.chain([cmd_list],[r_list],[elev_list],[azim_list],Pm_list,Pt_list,[h_list])
        merged_data=np.array(list(merged_data))
        res_list=np.vstack([res_list,merged_data])
        
        total_reward+=reward
        
        if done:
            res_list=np.delete(res_list,0,0)
            
            total_res.append(res_list)
            reward_list.append(total_reward)
            
            now = time.localtime()
            print ("%04d/%02d/%02d %02d:%02d:%02d" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec))
            print("episode : {} | final step : {} | total reward : {}".format(i_episode, t, total_reward.item()))
            break
            
        
    #목표 네트워크 업데이트, 모든 웨이트와 바이어스 복사
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.close()

2020/10/02 15:48:01
episode : 0 | final step : 201 | total reward : 202.0
2020/10/02 15:48:07
episode : 1 | final step : 201 | total reward : 202.0
2020/10/02 15:48:13
episode : 2 | final step : 201 | total reward : 202.0
2020/10/02 15:48:16
episode : 3 | final step : 98 | total reward : -2.0
2020/10/02 15:48:21
episode : 4 | final step : 201 | total reward : 202.0
2020/10/02 15:48:27
episode : 5 | final step : 201 | total reward : 202.0
2020/10/02 15:48:33
episode : 6 | final step : 201 | total reward : 202.0
2020/10/02 15:48:36
episode : 7 | final step : 98 | total reward : -2.0
2020/10/02 15:48:39
episode : 8 | final step : 98 | total reward : -2.0
2020/10/02 15:48:42
episode : 9 | final step : 98 | total reward : -2.0
2020/10/02 15:48:44
episode : 10 | final step : 99 | total reward : -1.0
2020/10/02 15:48:47
episode : 11 | final step : 99 | total reward : -1.0
2020/10/02 15:48:50
episode : 12 | final step : 98 | total reward : -2.0
2020/10/02 15:48:53
episode : 13 | final step : 9

2020/10/02 15:55:32
episode : 112 | final step : 201 | total reward : 202.0
2020/10/02 15:55:38
episode : 113 | final step : 201 | total reward : 202.0
2020/10/02 15:55:41
episode : 114 | final step : 99 | total reward : -1.0
2020/10/02 15:55:43
episode : 115 | final step : 99 | total reward : -1.0
2020/10/02 15:55:46
episode : 116 | final step : 99 | total reward : -1.0
2020/10/02 15:55:49
episode : 117 | final step : 98 | total reward : -2.0
2020/10/02 15:55:52
episode : 118 | final step : 98 | total reward : -2.0
2020/10/02 15:55:54
episode : 119 | final step : 98 | total reward : -2.0
2020/10/02 15:55:57
episode : 120 | final step : 98 | total reward : -2.0
2020/10/02 15:56:00
episode : 121 | final step : 98 | total reward : -2.0
2020/10/02 15:56:03
episode : 122 | final step : 99 | total reward : -1.0
2020/10/02 15:56:06
episode : 123 | final step : 99 | total reward : -1.0
2020/10/02 15:56:08
episode : 124 | final step : 98 | total reward : -2.0
2020/10/02 15:56:11
episode : 125 

2020/10/02 16:01:49
episode : 223 | final step : 98 | total reward : -2.0
2020/10/02 16:01:52
episode : 224 | final step : 99 | total reward : -1.0
2020/10/02 16:01:55
episode : 225 | final step : 98 | total reward : -2.0
2020/10/02 16:02:01
episode : 226 | final step : 201 | total reward : 202.0
2020/10/02 16:02:03
episode : 227 | final step : 98 | total reward : -2.0
2020/10/02 16:02:09
episode : 228 | final step : 201 | total reward : 202.0
2020/10/02 16:02:12
episode : 229 | final step : 99 | total reward : -1.0
2020/10/02 16:02:15
episode : 230 | final step : 98 | total reward : -2.0
2020/10/02 16:02:18
episode : 231 | final step : 99 | total reward : -1.0
2020/10/02 16:02:20
episode : 232 | final step : 98 | total reward : -2.0
2020/10/02 16:02:26
episode : 233 | final step : 201 | total reward : 202.0
2020/10/02 16:02:29
episode : 234 | final step : 98 | total reward : -2.0
2020/10/02 16:02:32
episode : 235 | final step : 98 | total reward : -2.0
2020/10/02 16:02:34
episode : 23

2020/10/02 16:08:41
episode : 333 | final step : 98 | total reward : -2.0
2020/10/02 16:08:44
episode : 334 | final step : 98 | total reward : -2.0
2020/10/02 16:08:47
episode : 335 | final step : 98 | total reward : -2.0
2020/10/02 16:08:50
episode : 336 | final step : 98 | total reward : -2.0
2020/10/02 16:08:53
episode : 337 | final step : 99 | total reward : -1.0
2020/10/02 16:08:56
episode : 338 | final step : 99 | total reward : -1.0
2020/10/02 16:08:58
episode : 339 | final step : 98 | total reward : -2.0
2020/10/02 16:09:01
episode : 340 | final step : 98 | total reward : -2.0
2020/10/02 16:09:04
episode : 341 | final step : 98 | total reward : -2.0
2020/10/02 16:09:10
episode : 342 | final step : 201 | total reward : 202.0
2020/10/02 16:09:13
episode : 343 | final step : 98 | total reward : -2.0
2020/10/02 16:09:19
episode : 344 | final step : 201 | total reward : 202.0
2020/10/02 16:09:25
episode : 345 | final step : 201 | total reward : 202.0
2020/10/02 16:09:28
episode : 34

2020/10/02 16:15:41
episode : 443 | final step : 98 | total reward : -2.0
2020/10/02 16:15:47
episode : 444 | final step : 201 | total reward : 202.0
2020/10/02 16:15:53
episode : 445 | final step : 201 | total reward : 202.0
2020/10/02 16:15:56
episode : 446 | final step : 98 | total reward : -2.0
2020/10/02 16:15:59
episode : 447 | final step : 98 | total reward : -2.0
2020/10/02 16:16:04
episode : 448 | final step : 201 | total reward : 202.0
2020/10/02 16:16:10
episode : 449 | final step : 201 | total reward : 202.0
2020/10/02 16:16:13
episode : 450 | final step : 98 | total reward : -2.0
2020/10/02 16:16:19
episode : 451 | final step : 201 | total reward : 202.0
2020/10/02 16:16:22
episode : 452 | final step : 98 | total reward : -2.0
2020/10/02 16:16:28
episode : 453 | final step : 201 | total reward : 202.0
2020/10/02 16:16:31
episode : 454 | final step : 99 | total reward : -1.0
2020/10/02 16:16:34
episode : 455 | final step : 99 | total reward : -1.0
2020/10/02 16:16:37
episod

2020/10/02 16:22:38
episode : 553 | final step : 201 | total reward : 202.0
2020/10/02 16:22:41
episode : 554 | final step : 99 | total reward : -1.0
2020/10/02 16:22:44
episode : 555 | final step : 98 | total reward : -2.0
2020/10/02 16:22:46
episode : 556 | final step : 98 | total reward : -2.0
2020/10/02 16:22:49
episode : 557 | final step : 99 | total reward : -1.0
2020/10/02 16:22:55
episode : 558 | final step : 201 | total reward : 202.0
2020/10/02 16:23:01
episode : 559 | final step : 201 | total reward : 202.0
2020/10/02 16:23:04
episode : 560 | final step : 99 | total reward : -1.0
2020/10/02 16:23:07
episode : 561 | final step : 98 | total reward : -2.0
2020/10/02 16:23:09
episode : 562 | final step : 98 | total reward : -2.0
2020/10/02 16:23:12
episode : 563 | final step : 98 | total reward : -2.0
2020/10/02 16:23:15
episode : 564 | final step : 98 | total reward : -2.0
2020/10/02 16:23:18
episode : 565 | final step : 98 | total reward : -2.0
2020/10/02 16:23:21
episode : 56

2020/10/02 16:29:38
episode : 663 | final step : 201 | total reward : 202.0
2020/10/02 16:29:41
episode : 664 | final step : 99 | total reward : -1.0
2020/10/02 16:29:44
episode : 665 | final step : 100 | total reward : 0.0
2020/10/02 16:29:49
episode : 666 | final step : 201 | total reward : 202.0
2020/10/02 16:29:52
episode : 667 | final step : 98 | total reward : -2.0
2020/10/02 16:29:55
episode : 668 | final step : 98 | total reward : -2.0
2020/10/02 16:29:58
episode : 669 | final step : 98 | total reward : -2.0
2020/10/02 16:30:04
episode : 670 | final step : 201 | total reward : 202.0
2020/10/02 16:30:07
episode : 671 | final step : 98 | total reward : -2.0
2020/10/02 16:30:09
episode : 672 | final step : 99 | total reward : -1.0
2020/10/02 16:30:12
episode : 673 | final step : 98 | total reward : -2.0
2020/10/02 16:30:18
episode : 674 | final step : 201 | total reward : 202.0
2020/10/02 16:30:21
episode : 675 | final step : 98 | total reward : -2.0
2020/10/02 16:30:24
episode : 

2020/10/02 16:36:25
episode : 773 | final step : 201 | total reward : 202.0
2020/10/02 16:36:28
episode : 774 | final step : 100 | total reward : 0.0
2020/10/02 16:36:30
episode : 775 | final step : 99 | total reward : -1.0
2020/10/02 16:36:36
episode : 776 | final step : 201 | total reward : 202.0
2020/10/02 16:36:39
episode : 777 | final step : 98 | total reward : -2.0
2020/10/02 16:36:42
episode : 778 | final step : 98 | total reward : -2.0
2020/10/02 16:36:48
episode : 779 | final step : 201 | total reward : 202.0
2020/10/02 16:36:51
episode : 780 | final step : 99 | total reward : -1.0
2020/10/02 16:36:54
episode : 781 | final step : 99 | total reward : -1.0
2020/10/02 16:37:00
episode : 782 | final step : 201 | total reward : 202.0
2020/10/02 16:37:05
episode : 783 | final step : 201 | total reward : 202.0
2020/10/02 16:37:08
episode : 784 | final step : 99 | total reward : -1.0
2020/10/02 16:37:11
episode : 785 | final step : 98 | total reward : -2.0
2020/10/02 16:37:17
episode 

In [ ]:
plt.figure(figsize=(15,10))
plt.xlabel("Episode")
plt.ylabel("Total rewards")
plt.plot(reward_list)

In [ ]:
def moving_average(data,n):
    len_data=len(data)-n
    res_data=np.zeros(len_data)

    for i in range(len_data):
        res_data[i]=sum(data[i:i+n])/n
    return res_data

In [ ]:
average_number=200
filtered_data=moving_average(reward_list,average_number)

In [ ]:
plt.figure(figsize=(15,10))
plt.xlabel("Episode")
plt.ylabel("Total rewards")
plt.plot(filtered_data)

In [ ]:
# plots
Deg2Rad = np.pi/180
Rad2Deg = 1/Deg2Rad

plt_res=total_res[0]


plt.figure(figsize=(15,9), dpi=100)

plt.subplot(511)
plt.plot(plt_res[:,0], label=r'$\dot{h}_{cmd}$')
plt.ylabel(r'$\dot{h}_{cmd}$ ($m/s$)'), plt.grid()

plt.subplot(512)
plt.plot(plt_res[:,10],label=r'$\{h}$')
plt.ylabel(r'$h$ (m)'), plt.grid()

plt.subplot(513)
plt.plot(plt_res[:,1],label=r'$\{r}$')
plt.ylabel(r'$r$ (m)'), plt.grid()

plt.subplot(514)
plt.plot(plt_res[:,2]*Rad2Deg, label='elevation')
plt.ylabel('elevation (deg)'), plt.grid()

plt.subplot(515)
plt.plot(plt_res[:,3]*Rad2Deg, label='azimuth')
plt.ylabel('azimuth (deg)'), plt.grid()

plt.legend()
plt.show()


In [ ]:
# trajectory plots

from mpl_toolkits.mplot3d import Axes3D

plt.figure(figsize=(12,9), dpi=100)
plt.gca(projection='3d')
plt.plot(plt_res[:,5], plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.xlim(-2000,2000)
plt.ylim(0,4000)
plt.legend()
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,5], plt_res[:,4], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('North')
plt.ylabel('Up')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

In [ ]:
# plots
Deg2Rad = np.pi/180
Rad2Deg = 1/Deg2Rad

plt_res=total_res[10]


plt.figure(figsize=(15,9), dpi=100)

plt.subplot(511)
plt.plot(plt_res[:,0], label=r'$\dot{h}_{cmd}$')
plt.ylabel(r'$\dot{h}_{cmd}$ ($m/s$)'), plt.grid()

plt.subplot(512)
plt.plot(plt_res[:,10],label=r'$\{h}$')
plt.ylabel(r'$h$ (m)'), plt.grid()

plt.subplot(513)
plt.plot(plt_res[:,1],label=r'$\{r}$')
plt.ylabel(r'$r$ (m)'), plt.grid()

plt.subplot(514)
plt.plot(plt_res[:,2]*Rad2Deg, label='elevation')
plt.ylabel('elevation (deg)'), plt.grid()

plt.subplot(515)
plt.plot(plt_res[:,3]*Rad2Deg, label='azimuth')
plt.ylabel('azimuth (deg)'), plt.grid()

plt.legend()
plt.show()

In [ ]:
# trajectory plots

from mpl_toolkits.mplot3d import Axes3D

plt.figure(figsize=(12,9), dpi=100)
plt.gca(projection='3d')
plt.plot(plt_res[:,5], plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.xlim(-2000,2000)
plt.ylim(0,4000)
plt.legend()
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,5], plt_res[:,4], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('North')
plt.ylabel('Up')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

In [ ]:
# plots
Deg2Rad = np.pi/180
Rad2Deg = 1/Deg2Rad

plt_res=total_res[100]


plt.figure(figsize=(15,9), dpi=100)

plt.subplot(511)
plt.plot(plt_res[:,0], label=r'$\dot{h}_{cmd}$')
plt.ylabel(r'$\dot{h}_{cmd}$ ($m/s$)'), plt.grid()

plt.subplot(512)
plt.plot(plt_res[:,10],label=r'$\{h}$')
plt.ylabel(r'$h$ (m)'), plt.grid()

plt.subplot(513)
plt.plot(plt_res[:,1],label=r'$\{r}$')
plt.ylabel(r'$r$ (m)'), plt.grid()

plt.subplot(514)
plt.plot(plt_res[:,2]*Rad2Deg, label='elevation')
plt.ylabel('elevation (deg)'), plt.grid()

plt.subplot(515)
plt.plot(plt_res[:,3]*Rad2Deg, label='azimuth')
plt.ylabel('azimuth (deg)'), plt.grid()

plt.legend()
plt.show()

In [ ]:
# trajectory plots

from mpl_toolkits.mplot3d import Axes3D

plt.figure(figsize=(12,9), dpi=100)
plt.gca(projection='3d')
plt.plot(plt_res[:,5], plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.xlim(-2000,2000)
plt.ylim(0,4000)
plt.legend()
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,5], plt_res[:,4], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('North')
plt.ylabel('Up')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

In [ ]:
# plots
Deg2Rad = np.pi/180
Rad2Deg = 1/Deg2Rad

plt_res=total_res[1000]


plt.figure(figsize=(15,9), dpi=100)

plt.subplot(511)
plt.plot(plt_res[:,0], label=r'$\dot{h}_{cmd}$')
plt.ylabel(r'$\dot{h}_{cmd}$ ($m/s$)'), plt.grid()

plt.subplot(512)
plt.plot(plt_res[:,10],label=r'$\{h}$')
plt.ylabel(r'$h$ (m)'), plt.grid()

plt.subplot(513)
plt.plot(plt_res[:,1],label=r'$\{r}$')
plt.ylabel(r'$r$ (m)'), plt.grid()

plt.subplot(514)
plt.plot(plt_res[:,2]*Rad2Deg, label='elevation')
plt.ylabel('elevation (deg)'), plt.grid()

plt.subplot(515)
plt.plot(plt_res[:,3]*Rad2Deg, label='azimuth')
plt.ylabel('azimuth (deg)'), plt.grid()

plt.legend()
plt.show()

In [ ]:
# trajectory plots

from mpl_toolkits.mplot3d import Axes3D

plt.figure(figsize=(12,9), dpi=100)
plt.gca(projection='3d')
plt.plot(plt_res[:,5], plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.xlim(-2000,2000)
plt.ylim(0,4000)
plt.legend()
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,5], plt_res[:,4], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('North')
plt.ylabel('Up')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

In [ ]:
# plots
Deg2Rad = np.pi/180
Rad2Deg = 1/Deg2Rad

plt_res=total_res[9999]


plt.figure(figsize=(15,9), dpi=100)

plt.subplot(511)
plt.plot(plt_res[:,0], label=r'$\dot{h}_{cmd}$')
plt.ylabel(r'$\dot{h}_{cmd}$ ($m/s$)'), plt.grid()

plt.subplot(512)
plt.plot(plt_res[:,10],label=r'$\{h}$')
plt.ylabel(r'$h$ (m)'), plt.grid()

plt.subplot(513)
plt.plot(plt_res[:,1],label=r'$\{r}$')
plt.ylabel(r'$r$ (m)'), plt.grid()

plt.subplot(514)
plt.plot(plt_res[:,2]*Rad2Deg, label='elevation')
plt.ylabel('elevation (deg)'), plt.grid()

plt.subplot(515)
plt.plot(plt_res[:,3]*Rad2Deg, label='azimuth')
plt.ylabel('azimuth (deg)'), plt.grid()

plt.legend()
plt.show()

In [ ]:
# trajectory plots

from mpl_toolkits.mplot3d import Axes3D

plt.figure(figsize=(12,9), dpi=100)
plt.gca(projection='3d')
plt.plot(plt_res[:,5], plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.xlim(-2000,2000)
plt.ylim(0,4000)
plt.legend()
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,5], plt_res[:,4], label='player', linewidth=3)
plt.plot(plt_res[:,8], plt_res[:,7], label='target', linewidth=3)
plt.xlabel('East')
plt.ylabel('North')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()

plt.figure(figsize=(12,9), dpi=100)
plt.plot(plt_res[:,4], -plt_res[:,6], label='player', linewidth=3)
plt.plot(plt_res[:,7], -plt_res[:,9], label='target', linewidth=3)
plt.xlabel('North')
plt.ylabel('Up')
plt.grid(), plt.legend(), plt.axis('equal')
plt.show()